<a href="http://www.insa-toulouse.fr/" ><img src="http://www.math.univ-toulouse.fr/~besse/Wikistat/Images/logo-insa.jpg" style="height:80px; display: inline"  alt="INSA"/></a> 

# Tutoriel en R: Détecter, Mesurer, Expliquer, Atténuer, la Discrimination Indirecte des Algorithmes d'Apprentissage Statistique

**Version courte**

**Résumé**
Analyse de données extraites d'un recensement aux USA de 1994 et disponibles sur le [dépôt de l'UCI](http://archive.ics.uci.edu/ml/). Elles permettent de mettre en relation le niveau de revenu (inférieur ou supérieur à 50k$), analogue à une "solvabilité" ou score de crédit, avec d'autres variables dont certaines sensibles car indiquant l'appartenance à un groupe protégé par la loi: genre, origine ethnique. Différents indicateurs de biais donc de sources de discrimination indirecte vis à vis d'un groupe, sont définis et illustrés sur ces données. Les principaux, consensuels dans la littérature, sont l'effet disproportionné ou *disparate / adverse impact* (DI) (*demographic equality*), le taux d'erreur conditionnel (*overall error equality*) et des mesures associées à l'asymétrie des matrices de confusion conditionnellement au groupe (*equalized odds*). Le tutoriel conduit a estimer ces différents biais lors de la prévision de la solvabilité par régression logistique (linéaire) en comparaison avec un algorithme de forêts aléatoires (non linéaire). La doctrine "officielle" des enquêtes par *testing*, adaptée pour détecter une discrimination individuelle directe, est également évaluée sur les prévisions de ces deux algorithmes. Une procédure élémentaire d'atténuation du biais systémique par *post-processing* est enfin exécutée afin d'évaluer son impact sur la précision de prévision et les autres biais.  

**Remarque**
- Les principaux résultats de ce tutoriel ont servi d'illustration pour une intervention lors d'un séminaire commun CNIL & Défenseur des Droits (05/2020); ils sont détaillés dans un article soumis ([Besse, 2020)](https://hal.archives-ouvertes.fr/hal-02616963)).
- Ce tutoriel peut être exécuté en local après chargement ou clônage du dépôt ou encore dans le nuage *Google Colab* en cliquant sur le lien ci-dessous:

<a href="https://colab.research.google.com/github/wikistat/Fair-ML-4-Ethical-AI/blob/master/AdultCensus/AdultCensus-R-biasDetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1 Introduction
### 1.1 Les données
Des données publiques disponibles sur le site [UCI repository](http://archive.ics.uci.edu/ml/) sont extraites de la base de données issue du recensement réalisé aux Etats Unis en 1994. Les deux fichiers *train* et *test* on été réunis en un seul. Ces données son largement utilisées et font référence comme outil de *benchmark* pour comparer les performances de méthodes d’apprentissage. L’objectif est de prévoir, avec plus ou moins de biais, la variable binaire "revenu annuel" supérieur ou inférieur à 50k$. Cette prévision n'impacte pas la personne mais comme la démarche et le contexte sont tout à fait similaires à ce que pourrait faire une banque pour évaluer un risque de crédit, cet exemple est très illustratif. Ce jeu de données est systématiquement utilisé (bac à sable) pour évaluer les propriétés des algorithmes d'apprentissage équitable ou loyal (*fair learning*) car, contrairement à beaucoup d'autres jeux de données utilisés pour cette fin (*e.g. german credit bank*), la vraie valeur de la variable cible est connue ainsi que l'origine ethnique des personnes concernées.

Dans les données initiales, 48.842 individus sont décrits par les 14 variables du tableau ci-dessous:


|Num|Libellé|Ensemble de valeurs|
|-|---------:|-------------------:|
|1|`Age`|real|
|2|`workClass`|Private, Self-emp-not-inc, Self-emp-inc, Federal-gov, Local-gov, State-gov, Without-pay, Never-worked|
|3|`fnlwgt`|real|
|4|`education`|Bachelors, Some-college, 11th, HS-grad, Prof-school, Assoc-acdm, Assoc-voc, 9th, 7th-8th, 12th, Masters, 1st-4th, 10th, Doctorate, 5th-6th, Preschool|
|5|`educNum`|integer|
|6|`mariStat`|Married-civ-spouse, Divorced, Never-married, Separated, Widowed, Married-spouse-absent, Married-AF-spouse|
|7|`occup`|Tech-support, Craft-repair, Other-service, Sales, Exec-managerial, Prof-specialty, Handlers-cleaners, Machine-op-inspct, Adm-clerical, Farming-fishing, Transport-moving, Priv-house-serv, Protective-serv, Armed-Forces|
|8|`relationship`|Wife, Own-child, Husband, Not-in-family, Other-relative, Unmarried
|9|`origEthn`|White, Asian-Pac-Islander, Amer-Indian-Eskimo, Other, Black|
|10|`sex`|Female, Male|
|11|`capitalGain`|real| 
|12|`capitalLoss`|real|
|13|`hoursWeek`|real|
|14|`nativCountry`|United-States, Cambodia, England, Puerto-Rico, Canada, Germany, Outlying-US(Guam-USVI-etc), India, Japan, Greece, South, China, Cuba, Iran, Honduras, Philippines, Italy, Poland, Jamaica, Vietnam, Mexico, Portugal, Ireland, France, Dominican-Republic, Laos, Ecuador, Taiwan, Haiti, Columbia, Hungary, Guatemala, Nicaragua, Scotland, Thailand, Yugoslavia, El-Salvador, Trinadad&Tobago, Peru, Hong, Holand-Netherlands|
|15|`income`|>50K, <=50K|


### 1.2 Préparation des données

Le traitement commence par une préparation détaillée des données:
- lecture et prise en charge des données en remarquant que la variable `fnlwgt` (Final sampling weight) a un [statut particulier](http://web.cs.wpi.edu/~cs4341/C00/Projects/fnlwgt) pas très clair; elle est éliminée;
- suppression des observations présentant des données manquantes, erreurs ou incohérences,
- regroupement des modalités très dispersées, 
- suppression des variables redondantes. 

### 1.3 Estimation des biais

Parmi tous les critères de biais existant susceptibles de mettre en évidence une discrimination indirecte (Zliobaitė, 2015), trois ont été privilégiés (cf. Vermat et Rubin, 2018): 
1. discrimination indirecte par l'effet disproportionné: *disparate impact* ou *demographic equality*
2. comparaison des taux d'erreur conditionnels: *overall error equality*
3. comparaison des rapports de cote: *conditional procedure accuracy equality* ou *disparate mistreatment* ou *equalized odds*.

L'accent est mis dans ce premier tutoriel sur l'estimation de l'effet disproportionné (*disparate* ou *adverse impact*) vis-à-vis du genre. L'approximation de l'intervalle de confiance (Besse et al. 2018) est comparée avec une estimation *bootstrap* conduisant aux mêmes résultats. Les estimations sont calculées sur les données initiales de la base (biais de société ou systémique) puis sur les prévisions du seuil de revenu obtenues par deux algorithmes (régression logistique et forêts aléatoires) pour évaluer le risque de discrimination. 

**Remarques** 
- un [calepin plus détaillé](https://github.com/wikistat/Fair-ML-4-Ethical-AI/blob/master/AdultCensus/AdultCensus-R-biasDetectionLong.ipynb) mais plus long propose une analyse exploratoire détaillée (AFCM) et une comparaison des impacts des autres algorithmes d'apprentissage et donc de leur effet discriminatoire fonction du biais de société (genre et origine ethnique). Cela permet d'en mieux comprendre les origines selon le déséquilibre des classes et surtout selon l'importance des interactions entre les variables. 
- le site [aif360](https://aif360.mybluemix.net/) propose également un ensemble d'exemples et tutoriels. Il est plus riche: autres jeux de données, autres critères et surtout plus d'algorithmes de débiaisage, mais présente soit des démonstrations triviales, soit des exemples de méthodes très sophistiquées d'atténuation du biais. Ce tutoriel se veut plus pédagogique pour appréhender pas-à-pas les problèmes. 

## 2 Exploration des données
Lors de cette phase de travail, il y a deux points de vue radicalement différents. 
- Celui illustré par Friedler et al. (2019) consiste à entraîner un algorithme sur les données brutes sans exploration "humaine" préalable faisant appel à des compétences statistiques; par principe, tout est automatisé.
- Celui proposé dans ce tutoriel est le résultat d'une démarche nécessitant des compétences élémentaires  statistiques pour explorer les données, comprendre leur structure, détecter les problèmes potentiels: données manquantes, atypiques, biais, classes rares, distributions "anormales"...) afin d'y remédier au mieux, et d'illustrer l'intérêt de l'objectif poursuivi.

Remarquons que ce deuxième point de vue de connaissance des données semble plus respectueux des [lignes directrices des experts de la CE pour une IA igne de confiance](https://ec.europa.eu/futurium/en/ai-alliance-consultation) et anticipe donc un futur réglement européen annoncé par le [livre blanc](https://ec.europa.eu/info/sites/info/files/commission-white-paper-artificial-intelligence-feb2020_fr.pdf).
### 2.1 Lecture et premières transformations
Deux possibilités pour charger les données issues du dépôt de l'UCI en fonction du mode d'exécution adopté; en local après avoir installé R ou à distance dans le nuage *Google Colab*. 
1. Dans le premier cas, les données sont chargées en même temps que le dépôt *Github*,
2. Dans le deuxième cas, <a href="https://colab.research.google.com/github/wikistat/Fair-ML-4-Ethical-AI/blob/master/AdultCensus/AdultCensus-R-biasDetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
les données et le programme de transformation sont chargés et lus dans l'environnement de *Google Colab*; il est nécessaire d'exécuter la cellule ci-dessous.

In [ ]:
# Exécuter cette cellule lors d'une exécution dans le nuage Google Colab
# sinon les données et fonctions sont déjà chargées en local
system("wget -P . https://github.com/wikistat/Fair-ML-4-Ethical-AI/raw/master/AdultCensus/adultTrainTest.csv")
system("wget -P . https://github.com/wikistat/Fair-ML-4-Ethical-AI/raw/master/AdultCensus/dataPrepAdultCensus.R")
system("mkdir ../Functions")
system("wget -P ../Functions https://raw.githubusercontent.com/wikistat/Fair-ML-4-Ethical-AI/master/Functions/dispImp.R")
system("wget -P ../Functions https://raw.githubusercontent.com/wikistat/Fair-ML-4-Ethical-AI/master/Functions/overErrEqual.R")
system("wget -P ../Functions https://raw.githubusercontent.com/wikistat/Fair-ML-4-Ethical-AI/master/Functions/oddsEqual.R")

In [ ]:
source("dataPrepAdultCensus.R")
dataBase = dataPrepAdultCensus()
summary(dataBase)

### 2.2 Description statistique élémentaire
Mettre en évidence des difficultés présentes sur certaines variables ou couples de variables.

In [ ]:
options(repr.plot.width=4, repr.plot.height=4)
hist(dataBase[,"LcapitalGain"],probability=T, main="",xlab="log(1+CapitalGain)")
boxplot(dataBase[,"LcapitalGain"], horizontal=TRUE,boxwex=.2,  outline=TRUE,  
        frame=F, col = "lightgrey", add = TRUE,at=0.2)

Nécessité de transformer certaines variables avant de contruire des modèles linéaires puis mise en évidence d'incohérences et de fortes redondances entre certaines variables.

In [ ]:
table(dataBase$relationship,dataBase$sex)   

In [ ]:
table(dataBase$education,dataBase$educNum)  

In [ ]:
table(dataBase$mariStat,dataBase$relationship)

In [ ]:
table(dataBase$origEthn,dataBase$nativCountry)

In [ ]:
mosaicplot(table(dataBase[,"origEthn"],dataBase[,"income"]),main="", col="lightblue",cex=1.3)

Quelques modifications sont apportées dans la base; des variables sont supprimées afin de ne garder qu'une seule présence d'une information sensible: genre et origine ethnique.
- Suppression de la variable 3 `fnlwgt` qui n'a guère de signification pour cette analyse.
- Création d'une variable binaire `Child`: présence ou non d'enfants.
- Suppression de la variable 8 `relationship` redondante avec le genre et le statut marital,
- Suppression de la variable 14 `nativCountry` redondante avec l'origine ethnique.
- La variable 9 `originEthn` est simplifiée à 2 classes: CaucYes *vs.* CaucNo

**Attention** Pour la suite, il est important que les niveaux des facteurs soient ordonnés de façon cohérente pour la bonne interprétation des tables de contingence et de leurs effectifs. Par convention, les modalités pré-jugées socialement "dévavorables": revenu faible, femme, non caucasien (codées 0), précèdent les autres: revenu élevé, homme, caucasien (codées 1). Il faut donc soit réordonner les niveaux de la variable `income` ou renommer les modalités pour faire correspondre l'ordre alphabétique; ce deuxième choix est exécuté.

In [ ]:
dataBase[,"child"]=as.factor(ifelse(dataBase$relationship=='Own-child',"ChildYes","ChildNo"))
dataBase[,"origEthn"]=as.factor(ifelse(dataBase$origEthn %in% c('Amer-Indian','Asian','Black','Other'),"CaucNo","CaucYes"))
dataBase[,"income"]=as.factor(ifelse(dataBase$income=='incLow',"incB","incH"))
datBas=dataBase[,-c(3,8,14)]
summary(datBas)

Quelques *mosaicplots* basiques montrent le lien des variables sensibles avec la cible (seuil de revenu) et mettent clairement en évidence le biais de société.

In [ ]:
mosaicplot(table(datBas[,"sex"],datBas[,"income"]),main="", col="lightblue",cex=1.3)

In [ ]:
mosaicplot(table(datBas[,"origEthn"],datBas[,"income"]),main="",col="lightblue",cex=1.3)

**Q** Commentaires sur les biais présents dans la base, les déséquilibres des classes.

### 2.3 [Analyse factorielle multiple des correspondances](http://wikistat.fr/pdf/st-m-explo-afcm.pdf)
Cette méthode permet d'avoir une vue d'ensemble de la structure des liaisons entre les variables qualitatives prises 2 à 2. C'est la raison pour laquelle les variables quantitatives ont été découpées en classes afin de mixer variables quantitatives découpées en classes et variables qualitatives

**Attention**: décommenter la commande d'installation ci-dessous lors d'une exécution dans le nuage ou si tout simplement cette librairie n'est pas installée.

In [ ]:
# install.packages("factoMineR")

In [ ]:
library(FactoMineR)
dataq=datBas[,c(2,3,5:10,12,15:17)]
afcm=MCA(dataq,graph=FALSE)
plot(1:30,afcm$eig[1:30,1],type="h")

Il pourrait être utile de considérer 3 ou 4 axes mais seuls deux sont utilisés dans cette approche élémentaire. 

**Remarque** Cette étape permettrait d'anonymiser les données pour des traitements ultérieurs "confidentiels". Les variables qualitatives suceptibles de ré-identifier les données sont remplacées par les composantes principales de l'ACP des vecteurs profils lignes du tableau disjontif complet. Les liens entre individus sont conservés sans possibilité de les identifier.

In [ ]:
options(repr.plot.width=8, repr.plot.height=8)
plot.MCA(afcm,invisible=c("ind"),habillage="quali")

Remarquer la très forte et attendue liaison entre niveau d'éducation et activité selon un axe de dispersion suivant la première bissectrice, lui même en relation avec les modalités haut et bas revenus alors que le genre, en première approximation, semble orthogonal à cet axe mais en relation avec le type d'activité et le nombre d'heures de travail par semaine.

Ce graphique laisse penser que le lien entre genre et niveau de revenu ne serait pas dû à une sous qualification des femmes mais à une moindre activité (travail partiel) ou à des types d'emplois moins rémunérés.

Cette représentation est complétée en projetant les individus colorés selon la variable revenu. De façon triviale, les hauts revenus se concentrent du côté des niveaux d'éducation les plus élevés.

*En résumé*: un lien direct entre revenu et niveau d'éducation, pas nécessairement de lien entre genre et niveau d'éducation mais un "frein" au revenu lié au type de travail ou d'occupation : administratif, service... et à de possibles temps partiels.

In [ ]:
plot(afcm$ind$coord,type="p",pch=".",cex=2,col=as.integer(dataBase$income))

### 2.4 Préparation des échantillons
La base de données est divisée en deux échantillons d'apprentissage et de test. 

In [ ]:
summary(datBas)

Sélection des variables utilisées par la suite, celles qualitatives issues des quantitatives ne sont pas reprises.

In [ ]:
datBas=datBas[,c("age","educNum","mariStat","occup","origEthn",
                 "sex","hoursWeek","income","LcapitalGain","LcapitalLoss","child")]
summary(datBas)
write.csv(datBas,file="adultBase-R.csv",row.names = FALSE)

Extraction des échantillons d'apprentissage et de test. Vous pouvez changer la valeur initiale `seed` du générateur de nombre aléatoires afin d'obtenir une autre répartition entre apprentissage et test.

In [ ]:
set.seed(11) # initialisation du générateur
# Extraction des échantillons
test.ratio=.2   # part de l'échantillon test
npop=nrow(datBas) # nombre de lignes dans les données
nvar=ncol(datBas) # nombre de colonnes
# taille de l'échantillon test
ntest=ceiling(npop*test.ratio) 
# indices de l'échantillon test
testi=sample(1:npop,ntest)
# indices de l'échantillon d'apprentissage
appri=setdiff(1:npop,testi) 

In [ ]:
# construction de l'échantillon d'apprentissage
datApp=datBas[appri,]
# construction de l'échantillon test 
daTest=datBas[testi,]
# sauvegarde des fichiers pour le deuxième tutoriel
write.csv(datApp,"datApp.csv")
write.csv(daTest,"daTest.csv")

Définition d'une fonction pour calculer le taux d'erreur usuel de la matrice de confusion. Avec l'ordonnancement des modalités, les bien classés sont les termes diagonaux de la matrice de confusion.

In [ ]:
tauxErr=function(table){round((table[1,2]+table[2,1])/sum(table)*100,2)}

Les différents résultats sont stockés dans un tableau dans l'objectif d'un graphe comparatif de synthèse.

In [ ]:
matRes=data.frame(matrix(0,7,4))
rownames(matRes)=c("dataBaseBias","linLogit","tree","linLogit_w_S","testingLogit","randomForest","RFdiscrPos")
colnames(matRes)=c("Lower","DI","Upper","Accuracy")

## 3 Prévision du seuil de revenu 
Une comparaison détaillée (cf. le [tutoriel](https://github.com/wikistat/Apprentissage/blob/master/Adult-Census/Apprent-Python-AdultCensus.ipynb)) de la plupart des modèles et algorithmes de prévision du seuil de revenu en fonction des différentes variables met en évidence des résultats de prévision un peu meilleurs obtenus par l'algorithme de *gradient boosting* (version xtrem). Néanmoins, on peut se limiter ici à un choix plus restreint de modèles et algorithmes pour comprendre l'impact sur la discrimination entre:
- la régression logistique (linéaire) interprétable,
- les forêts aléatoires (non linéaire intégrant les interactions) mais sans capacité d'interprétation.

Un arbre binaire conduit à des qualités de prévision intermédiaires entre régression logistique et forêt aléatoire mais n'est pas facilement interprétable à cause du nombre jugé "optimal" mais élevé de feuilles.

### 3.1 Prévision par [régression logistique](http://wikistat.fr/pdf/st-m-app-rlogit.pdf)

In [ ]:
# estimation du modèle complet
log.lm=glm(income~.,data=datApp,family=binomial)
# significativité des paramètres
anova(log.lm,test="Chisq")

**Q** Commentaires sur la significativité des paramètres.

In [ ]:
summary(log.lm)

**Q** Interprétation du rôle des variables sur le seuil de revenu.

In [ ]:
# Prévision de l'échantillon test
pred.log=predict(log.lm,newdata=daTest,type="response")
# Matrice de confusion pour la prévision du 
# dépassement de seuil
confMat=table(pred.log>0.5,daTest$income)
confMat

In [ ]:
err=tauxErr(confMat)
matRes[2,4]=(100-err)/100
err

**Remarque** Un modèle de régression logistique avec interactions, donc quadratique, ne conduit pas à une  prévision significativement meilleure mais nécessite une sélection de variables (*e.g. stepwise* ou `both`) longue à exécuter. Toujours par souci de concision, elle n'est pas reproduite; consulter le [calepin plus complet](https://github.com/wikistat/Fair-ML-4-Ethical-AI/blob/master/AdultCensus/AdultCensus-R-biasDetectionLong.ipynb).

### 3.2 Prévision par arbre binaire

In [ ]:
summary(datApp)

**Attention**: décommenter la commande d'installation ci-dessous lors d'une exécution dans le nuage ou si tout simplement cette librairie n'est pas installée.

In [ ]:
# install.packages("rpart")

In [ ]:
library(rpart)
# Estimation du modèle complet
tree.mod=rpart(income~.,data=datApp,cp=0.0001)

Optimisation de la pénalité

In [ ]:
xmat=xpred.rpart(tree.mod)
xerr=(xmat-as.integer(datApp[,"income"]))^2
CVerr=apply(xerr,2,sum)

In [ ]:
as.numeric(attributes(which.min(CVerr))$names)

In [ ]:
tree.opt=rpart(income~.,data=datApp,control=rpart.control(cp=as.numeric(attributes(which.min(CVerr))$names)))

In [ ]:
library(partykit)
options(repr.plot.width=16, repr.plot.height=16)
plot(as.party(tree.opt), type="simple")

**Q** Que dire de la clareté de l'interprétation de ce modèle?

**Q** Que dire de la présence de la variable genre?

In [ ]:
pred.tree=predict(tree.opt,newdata=daTest,type="class")
confMat=table(pred.tree,daTest$income)
confMat
err=tauxErr(confMat)
matRes[3,4]=(100-err)/100
err

*Remarque* un modèle plus simple est obtenu en augmentant la pénalisation mais avec pour conséquence de restreindre les performances.

### 3.3 Prévision par [forêts aléatoires](http://wikistat.fr/pdf/st-m-app-agreg.pdf)

**Q** Quelles sont les options par défaut utilisées ci-dessous?

In [ ]:
# install.packages("randomForest")

In [ ]:
library(randomForest)
rf.mod=randomForest(income~.,data=datApp)
pred.rf=predict(rf.mod,newdata=daTest,type="response")
confMat=table(pred.rf,daTest$income)
confMat
err=tauxErr(confMat)
matRes[6,4]=(100-err)/100
err

**Q** Comparer les résultats obtenus, précision et explicabilité.

Les calculs peuvent également être conduits en considérant l'origine ethnique comme variable sensible (consulter le [calepin plus complet](https://github.com/wikistat/Fair-ML-4-Ethical-AI/blob/master/AdultCensus/AdultCensus-R-biasDetectionLong.ipynb)) mais les résultats sont moins nets, moins "pédagogiques".

### 3.4 Modèle de régression logistique sans la variable genre
Une approche très naïve pour construire un apprentissage "loyal" consiste à supprimer la variable sensible. Le modèle de régression logistique est donc estimé sans cette variable afin d'évaluer par la suite l'impact sur le biais.

In [ ]:
# estimation du modèle 
log_g.lm=glm(income~.,data=datApp[,-6],family=binomial)

In [ ]:
# Prévision
pred_g.log=predict(log_g.lm,newdata=daTest[,-8],type="response")
# Matrice de confusion pour la prévision du 
# dépassement de seuil
confMat=table(pred_g.log>0.5,daTest$income)
confMat

In [ ]:
err=tauxErr(confMat)
matRes[4,4]=(100-err)/100
err

**Q** que dire de la qualité de prévision sans la variable genre?

## 4 Estimation de l'effet disproportionné
### 4.1 Définition
Les mesures de discrimination indirecte ou de groupe sont basées sur un critère d'effet disproportionné ou *disparate / adverse impact* (*DI*) apparu aux USA en 1971 (Barocas et Selbst, 2016) pour détecter des discriminations à l'embauche. Ce critère est défini par le rapport de deux probabilités. La probabilité de bénéficier d'une situation ou décision favorable (revenu élevé, crédit, emploi, logement...) pour une personne du groupe protégé par la loi: femme ou origine non caucasienne, sur la même probabilité pour une personne de l'autre groupe: homme ou origine caucasienne.

*Notations*: 

- $Y$ est la variable cible ou à expliquer, ici le revenu, $Y=1$ revenu élevé *vs* $Y=0$; 
- $g(X)=\hat{Y}$ le score élevé ou faible ou prévision du revenu, $g(X)=\hat{Y}=0$: prévision d'un revenu faible;
- $S$ est la variable sensible qui désigne le groupe en principe protégé par la loi contre une possible discrimination. Il s'agit ici du genre masculin ($S=1$) ou féminin ($S=0$). 

L'effet disproportionné mesure une situation de biais de société déjà présent dans la base de données. 
$$DI=\frac{P(Y=1|S=0)}{P(Y=1|S=1)}.$$
Il est estimé à partir des valeurs de la table de contingence croisant les variables $Y$ et $S$ par le rapport:
$$\frac{n_{21}}{(n_{11}+n_{21})}/\frac{n_{22}}{(n_{12}+n_{22})}.$$

Appliqué à la prévision $g(X)=\hat{Y}$ de la variable cible $Y$, il mesure le biais de cette prévision et donc le risque de discrimination opéré par la prévision.

### 4.2 Fonction `dispImp`
Une fonction R calcule le $DI$ et fournit de plus une estimation par intervalle de confiance (Besse et al. 2018) décomposant la fonction de densité de la statistique de test par la *delta method*. Cette fonction a trois arguments:   
- la variable $S$ considérée comme sensible: un facteur à deux niveaux ordonnés "défavorable" puis "favorable";
- la variable cible $Y$ ou sa prévision $g(X)=\hat{Y}$: également un facteur à deux niveaux défavorable puis favorable;
- le risque de l'intervalle de confiance, par défaut 5%.

Cette fonction renvoie les trois estimations $DI$ et $IC_g$, $IC_d$ bornes de l'intervalle de confiance.

Morris S., Lobsenz R. (2000) avait déjà suggéré de calculer une estimation du *DI* par intervalle de confiance mais en faisant l'hypothèse de distributions gaussiennes; cette approximation n'est pas justifiée pour les effectifs d'une table de contingence!

In [ ]:
source("../Functions/dispImp.R")

### 4.3 Effet disproportionné ou biais de la base d'apprentissage
La fonction `dispImp` nécessite que les niveaux des facteurs soient dans l'ordre lexicographique: niveaux "défavorable" puis "favorable".

Table de contingence croisant $Y$ (revenu) avec $S$ (genre).

In [ ]:
tableDI=table(datBas$income,datBas$sex)
tableDI

*Estimation ponctuelle* de $DI=\frac{n_{21}}{(n_{11}+n_{21})}/\frac{n_{22}}{(n_{12}+n_{22})}.$

In [ ]:
round((tableDI[2,1]/(tableDI[1,1]+tableDI[2,1]))/(tableDI[2,2]/(tableDI[1,2]+tableDI[2,2])),3)

*Estimation par intervalle de confiance* approché par *delta method*. 

In [ ]:
round(dispImp(datBas[,"sex"],datBas[,"income"]),3)

**Q** Commenter le biais ainsi mesuré, comparer avec le graphique obtenu (*mosaic plot*) lors de l'exploration.

*[Estimation bootstrap](http://wikistat.fr/pdf/st-m-app-bootstrap.pdf) de l'intervalle de confiance*

L'estimation de l'intervalle de confiance est comparée avec le comportement du *DI* sur des échantillons *bootstrap* (Efron 1987).

In [ ]:
B=1000 
set.seed(11)
n=nrow(datBas)
res=matrix(0,B,3)
for (i in 1:B)
    {
    boot=sample(n,n,replace=T)
    res[i,]=dispImp(datBas[boot,"sex"],datBas[boot,"income"])
    }

In [ ]:
options(repr.plot.width=5, repr.plot.height=4)
DI_confInt_delta <- round(dispImp(datBas[,"sex"], datBas[,"income"]), 3)
plot(res[,2],ylim=c(0.3,0.45),pch='.')
lines(res[,2],col=3,pch='.')
abline(h=DI_confInt_delta[c(1, 3)], col=2) 

**Q** Que dire des estimations du *DI* sur les échantillons bootstrap par rapport aux bornes de l'intervalle de confiance ?

Une fonction de la librairie `boot` fournit une estimation bootstrap de l'intervalle de confiance.

In [ ]:
library(boot)
fc <- function(d, i){ 
    d2 <- d[i,]
    return(statistic=dispImp(d2$sex,d2$income)[2])
}
set.seed(11)
bootDI <- boot(datBas,fc, R=1000)
bootDI

In [ ]:
boot.ci(boot.out = bootDI, type = "perc")

**Q** Comparer l'approximation par delta method et l'estimation bootstrap de intervalle de confiance.

**Q** Compte tenu des temps de calcul, que privilégier?

L'intervalle est finalement estimé sur l'échantillon test afin de comparer avec ses différentes prévisions.

In [ ]:
ic=round(dispImp(daTest[,"sex"],daTest[,"income"]),3)
matRes[1,]=c(ic, 1)
ic

**Q** Que dire de la taille de l'intervalle de confiance?

### 4.5 Effet disproportionné des prévisions 
Le même ratio ou impact disproportionné calculé sur les prévisions $g(X)$ de $Y$ plutôt que sur $Y$ mesure explicitement l'effet de la prévision. Il revient à tester l'égalité des taux de prévision favorable entre les deux groupes. 

La valeur seuil de la probabilité pour prédire le niveau de revenu est fixée par défaut à $0.5$.

#### Régression logistique

In [ ]:
Yhat=as.factor(pred.log>0.5)

In [ ]:
ic=round(dispImp(daTest[,"sex"],Yhat),3)
matRes[2,1:3]=ic
ic

#### Arbre binaire

In [ ]:
ic=round(dispImp(daTest[,"sex"],pred.tree),3)
matRes[3,1:3]=ic
ic

#### *Random Forest*

In [ ]:
ic=round(dispImp(daTest[,"sex"],pred.rf),3)
matRes[6,1:3]=ic
ic

**Q** Comparer les trois intervalles de confiance de l'estimation du *DI* dans le cas des données initiales, de la prévision par régression logistique et de celle par forêt aléatoire. Conclusion?

#### Effet disproportionné des prévisions sans la variable sensible genre

In [ ]:
Yhat_g=as.factor(pred_g.log>0.5)
ic=round(dispImp(daTest[,"sex"],Yhat_g),3)
matRes[4,1:3]=ic
ic

**Q** Que devient le DI avec une prévision n'utilisant pas la variable sensible?

Le [calepin long](https://github.com/wikistat/Fair-ML-4-Ethical-AI/blob/master/AdultCensus/AdultCensus-R-biasDetectionLong.ipynb) vérifie ces résultats en considérant (validation croisée *Monte Carlo*) 20 réplications de la séparation des échantillons d'apprentissage et de test sur lesquels sont entraînés trois algorithmes: logistique linéaire, arbre de décision, *random forest*, avant d'évaluer le DI observé sur la prévision de l'échantillon test. 

Comme attendu, la précision dépend fortement de l'algorithme choisi mais aussi, c'est important et moins attendu, de l'estimation du *DI*. De façon relativement contre-intuitive, meilleure est la précision et moins le biais est renforcé par rapport à celui `DIbase` des données d'entraînement. Mais, pour un algorithme donné, le *DI* n'est pas corrélé à la précision sur un échantillon d'apprentissage.

**Q** Conclusion partielle sur l'impact de chaque algorithme, dont il faudrait aussi prendre en compte les capacités d'interprétabilité, surtout vis-à-vis de la loi américaine.

**Attention** comme le rappelle Friedler et al. (2019) les résultats et conclusions peuvent changer d'un jeu de données à un autre. C'est déjà bien connu en ce qui concerne la précision de la prévision, il est nécessaire de l'intégrer dans la gestion du biais. Les résultats présentés dans ce tutoriel, sont sur certains points, différents de ceux de Friedler et al. (2019). La principale raison en est sans doute la différence de stratégie adoptée pour le traitement préalable des données. Par principe, Friedler et al. (2019) analysent sans aucune perspective statistique élémentaire les données brutes et donc sans aucun traitement préalable. Il y a peut être aussi, à vérifier, des différences d'implémentation entre les versions R et Python des algorithmes.

### 4.6 Effet disproportionné *vs. Testing*
#### Enquête commanditée par la DARES
Le *testing* est, à l'origine, une méthode usuelle permettant de détecter une discrimination *directe* opérée par un humain. Elle est "adaptée" (Riach et Rich 2002) et déployée par la ([DARES](http://dares.travail-emploi.gouv.fr/dares-etudes-et-statistiques/etudes-et-syntheses/dares-analyses-dares-indicateurs-dares-resultats/testing)) du Ministère du Travail  (cf. [article Le Monde 2020](https://www.lemonde.fr/societe/article/2020/01/08/une-etude-montre-des-discriminations-a-l-embauche-significatives-en-fonction-de-l-origine_6025227_3224.html)) à la détection par enquête d'une discrimination indirecte envers un groupe. Il consiste à évaluer la variabilité d'une décision lorsque seule la modalité de la variable sensible est modifiée. 

Les calculs ci-dessous permettent de reproduire les résultats globaux de la dernière enquête de la DARES.

In [ ]:
origine.i=matrix(0,10000,1);reponse.i=matrix(0,10000,1)
origine.i[4536:8910]=1;origine.i[9376:10000]=1
reponse.i[8911:10000]=1
origine=factor(origine.i,labels=c("Maghreb","France"))
reponse=factor(reponse.i,labels=c("Negative","Positive"))
table(reponse, origine)

In [ ]:
100*465/5000;100*625/5000; 465/625

Les ratios sont bien ceux de l'enquête. Il y a de très nombreuses façons de les comparer afin de conclure ou non à une discirmination jugée significative. Que donne l'évaluation de l'effet disproportionné?

In [ ]:
options(repr.plot.width=4, repr.plot.height=4)
mosaicplot(table(origine,reponse),main="",col="lightblue",cex=1.3)

In [ ]:
round(dispImp(origine,reponse),3)

**Q** Cette enquête par *testing* met-elle en évidence une discrimination statistiquement significative au regard du règlement américain (règle des 4/5ème)?

**Q** Que dire de la précision de l'évaluation du *DI* avec 10 000 CVs envoyés? Serait-il possible de conclure pour une entreprise donnée?

#### *Testing* d'un algorithme d'apprentissage
Que se passe-t-il si le *testing* est appliqué à une décision automatique pilotée par un algorithme d'apprentissage? 

Les prévisions de revenu sont calculées pour les mêmes personnes de l'échantillon test en prenant en compte le genre initial puis le genre opposé. Dans ce cas, une femme pour laquelle la prévision de revenu ou solvabilité change lorsque la variable genre passe de `Female` à `Male` serait en droit d'intenter une action en justice pour discrimination directe. 

In [ ]:
daTest2=daTest
# Changement de genre
daTest2$sex=as.factor(ifelse(daTest$sex=="Male","Female","Male"))
# Prévision du "nouvel" échantillon test
pred2.log=predict(log.lm,daTest2,type="response")
Yhat2=as.factor(pred2.log>0.5)

In [ ]:
table(Yhat,Yhat2)

In [ ]:
# répartition par genre
table(Yhat,Yhat2,daTest$sex)

**Q** Compléter: Il y a $x+y$ personnes dont la prévision de revenu change lorsqu'elles changent de genre. Et le changement est dans le sens attendu.
- $x$ femmes passent d'une prévision de revenu faible à une prévision de revenu élevé
- $y$ hommes prennnent le chemin inverse  lorsqu'ils deviennent des femmes; ces hommes bénificiaient donc d'une discrimination positive!

Quels résultats fourniraient une enquête par testing consistant à envoyer deux fois les "dossiers" à l'algorithme, une fois pour chaque genre? Il faut donc considérer deux fois plus de personnes en concaténant les deux prévisions. Ceci conduit à la table de contingence ci-dessous. 

In [ ]:
# fonction de concaténation de deux vecteurs de type factor
c.factor <- function(..., recursive=TRUE) unlist(list(...), recursive=recursive) 
Yhat=factor(Yhat,labels=c("incB","incH")); Yhat2=factor(Yhat2,labels=c("incB","incH"))
mosaicplot(table(c.factor(daTest$sex,daTest2[,"sex"]),c.factor(Yhat,Yhat2)),main="",col="lightblue",cex=.8)

In [ ]:
ic=round(dispImp(c.factor(daTest$sex,daTest2[,"sex"]),c.factor(Yhat,Yhat2)),3)
matRes[5,1:3]=ic
ic

**Q** Conclusion: le testing est-il adapté à détecter une disrimination algorihtmique?

**Q** A quoi est principalement due cette incapacité? Penser au rôle de la variable genre dans les prévisions.

#### *Testing* et prévision "déloyale" 
Une façon simple pour une entreprise de se prémunir contre une opération de *testing* consiste à fixer comme probablité prédite le maximum des deux probablités obtenues en échangeant les modalités de la variable sensible. De façon générale, choisir la situation la plus favorable pour la personne quelque soit le genre observé. La discrimination individuelle repérable par *testing* est neutralisée et l'influence sur le taux d'erreur quasi négligeable. 

In [ ]:
fairPredictGenre=pmax(pred.log, pred2.log) 
confMat=table(fairPredictGenre>0.5,daTest$income)
confMat;err=tauxErr(confMat)
matRes[5,4]=(100-err)/100
err

In [ ]:
round(dispImp(daTest$sex,as.factor(pred.log>0.5)),3)
round(dispImp(daTest$sex,as.factor(fairPredictGenre>0.5)),3)

**Attention** Cette procédure masque de façon **intentionnelle** la discrimination directe détectable tout en favorisant une discrimination indirecte, elle est clairement **condamnable au sens du code pénal**. Soyez prudent dans vos futures pratiques professionnelles!

## 5 Expliquer, atténuer la discrimination ?
De façon notable, le modèle de régression logistique reproduit le biais social et le renforce en introduisant donc de la discrimination; c'est moins net pour l'algorithme des forêts aléatoires,  Est-il possible d'expliquer ce comportement ou plus précisément d'utiliser le bon modèle ou bon algorithme qui l'évite? Le [calepin long](https://github.com/wikistat/Fair-ML-4-Ethical-AI/blob/master/AdultCensus/AdultCensus-R-biasDetectionLong.ipynb) compare différents algorithmes selon diverses situations notamment en attribuant sans succès plus de poids aux femmes afin de compenser leur sous-représentation. 

La littérature propose une avalanche de méthodes pour débiaiser une décision algorithmique. Trois approches sont développées:
- *Pre-processing* en débiaisant les données d'apprentissage;
- *Processing* en pénalisant la fonction objectif avec une contrainte d'équité mais l'optimisation n'est plus convexe;
- *Post-processing* en dé-biaisant les décisions.

Friedler et al. (2019) et le site [AIF360](https://aif360.mybluemix.net/) proposent une comparaison numérique systématique de quelques unes de ces approches sur plusieurs jeux de données publiques donc celui de ce tutoriel. Les codes de Friedler et al. (2019) sont disponibles sur un dépôt et une version "simplifiée" est accessible dans un [autre tutoriel](https://github.com/wikistat/Fair-ML-4-Ethical-AI/blob/master/AdultCensus/AdultCensus-Python-BiasDetectionBiblio.ipynb) en python qui focalise les calculs sur ce seul jeu de données.

Une version rudimentaire mais efficace de post-processing consiste à estimer deux modèles ou entraîner deux algorithmes, l'un pour les femmes et l'autre pour les hommes puis d'adapter le seuil de décision afin de réduire l'effet disproportioné tout en contrôlant le taux d'erreur. Cette procédure est testée dans les deux cas de la régression logistique et des forêts aléatoires. C'est une façon d'introduire une dose de discrimination positive afin d'infléchir vers plus d'équité sociale.

Une première partie consiste à estimer séparément les modèles avant d'introduire de la discrimination positive dans une deuxième partie.

**Remarque** Il n'est sans doute pas nécessaire d'estimer deux modèles de forêts aléatoires selon le genre. Le seul post-processing des seuils de décision devrait suffire. Le choix de deux modèles par genre est opéré, surtout pour la régression logistique, afin de prendre en compte l'effet important des interactions dans le modèle. 


### 5.1 Séparation des deux échantillons
Les échantillons sont séparés en deux parties.

In [ ]:
datAppF=subset(datApp, sex == 'Female') 
datAppM=subset(datApp, sex == 'Male')
daTestF=subset(daTest, sex == 'Female')
daTestM=subset(daTest, sex == 'Male')
summary(datAppM)

### 5.2 Régression logistique
Estimation des deux modèles.

In [ ]:
reg.log=glm(income~.,data=datApp,family=binomial)
# estimation des deux modèles
reg.logF=glm(income~.,data=datAppF[,-6],family=binomial)
reg.logM=glm(income~.,data=datAppM[,-6],family=binomial)
# comparaison des paramètres
summary(reg.logF);summary(reg.logM)

**Q** La comparaison des paramètres des deux modèles apporte-t-elle des informations?

In [ ]:
# prévision des modèles
yHat=predict(reg.log,newdata=daTest,type="response")
yHatF=predict(reg.logF,newdata=daTestF,type="response")
yHatM=predict(reg.logM,newdata=daTestM,type="response")
# compilation des prévisions
yHatFM=c(yHatF,yHatM)

In [ ]:
daTestFM=rbind(daTestF,daTestM)
dim(daTestFM)

In [ ]:
# erreurs cumulées
table(yHatFM>0.5,daTestFM$income)

In [ ]:
table(yHat>0.5,daTest$income)

In [ ]:
tauxErr(table(yHatFM>0.5,daTestFM$income))

In [ ]:
tauxErr(table(yHat>0.5,daTest$income))

**Q** Que devient l'erreur de prévision une fois que les deux modèles sont associés avec le même seuil de décision?

In [ ]:
# biais cumulés vs. biais 
round(dispImp(daTestFM[,"sex"],as.factor(yHatFM>0.5)),3); round(dispImp(daTest[,"sex"],as.factor(yHat>0.5)),3)

In [ ]:
# Rappel: Biais de la base de test
round(dispImp(daTestFM[,"sex"],daTestFM[,"income"]),3)

**Q** Que devient le bais?

### 5.3 *Random Forest*

In [ ]:
library(randomForest)
# Modèle initial
RFinit=randomForest(income~.,data=datApp)
# Modèle en changeant les poids
RFinitW=randomForest(income~.,data=datApp,weigth=w)
# estimation des deux modèles
RF.F=randomForest(income~.,data=datAppF[,-6])
RF.M=randomForest(income~.,data=datAppM[,-6])
# comparaison des paramètres

In [ ]:
# prévision des modèles
yHatrf=predict(RFinit,newdata=daTest,type="response")
yHatrfW=predict(RFinitW,newdata=daTest,type="response")
yHatFrf=predict(RF.F,newdata=daTestF,type="response")
yHatMrf=predict(RF.M,newdata=daTestM,type="response")
# compilation des prévisions
yHatFMrf=c(yHatFrf,yHatMrf)

In [ ]:
# erreurs cumulées vs. erreur initiale de RF
table(yHatFMrf,daTestFM$income); table(yHatrf,daTest$income)

**Q** Comparer les erreurs.

In [ ]:
tauxErr(table(yHatFMrf,daTestFM$income))
tauxErr(table(yHatrf,daTest$income))
tauxErr(table(yHatrfW,daTest$income))

In [ ]:
# Biais cumulé vs. biais du modèle initial vs. biais de la base
round(dispImp(daTestFM[,"sex"],as.factor(yHatFMrf)),3)
round(dispImp(daTest[,"sex"],as.factor(yHatrf)),3)
round(dispImp(daTest[,"sex"],as.factor(yHatrfW)),3)
round(dispImp(daTestFM[,"sex"],daTestFM[,"income"]),3)

**Q** Comparer les biais.
### 5.4 Atténuation de la discrimination par *post-processing*
La procédure (*post-processing*) ci-dessous est la plus simple. Elle consiste à introduire une forme de discrimination positive en intervenant sur le seuil de décision pour les femmes tout en conservant celui de $0.5$ pour les hommes. Elle est appliquée dans ce tutoriel au seul algorithme de forêt aléatoire. Une procédure graphique d'optimisation a été appliquée mais non reproduite afin de contrôler, sur l'échantillon test initial, l'effet de la correction à la fois sur le biais et sur l'erreur de prévision. Le seuil choisi ($0.7$) pour les femmes pour décider d'un revenu faible ($<50$k$) en découle. *Attention* ce seuil peut dépendre de l'échantillon test et donc de l'initialisation du générateur de nombres aléatoires. Une procédure de validation croisée serait bienvenue!

Par ailleurs, il n'est sans doute pas nécessaire de considérer deux modèles un pour les femmes, un pour les hommes, l'intervention d'un seuil conditionnel au genre devrait suffire. La construction de deux modèles selon le genre a surtout été introduite pour intégrer les effets interactions en régression logistique.

In [ ]:
# prévision des modèles en modifiant le seuil des femmes
yHatFrfDP=predict(RF.F,newdata=daTestF,type="prob")[,1]<0.7
yHatMrfDP=predict(RF.M,newdata=daTestM,type="prob")[,1]<0.5
# compilation des prévisions
yHatFMrfDP=c(yHatFrfDP,yHatMrfDP)

In [ ]:
table(yHatFMrfDP,daTestFM$income)

In [ ]:
err=tauxErr(table(yHatFMrfDP,daTestFM$income))
matRes[7,4]=(100-err)/100
err

**Q** Que devient l'erreur de prévision?

In [ ]:
ic=round(dispImp(daTestFM[,"sex"],as.factor(yHatFMrfDP)),3)
matRes[7,1:3]=ic
ic

**Q** Que devient le biais?
### 5.5 Synthèse graphique des résultats
Les résultats précédents: précisions des prévisions et intervalles de confiance sont collectés et affichés sur le même graphique.

In [ ]:
matRes

**Attention**: décommenter la commande d'installation ci-dessous lors d'une exécution dans le nuage ou si tout simplement cette librairie n'est pas installée.

In [ ]:
# install.packages("Publish")
library(Publish)
options(repr.plot.width=10, repr.plot.height=5)
plotConfidence(x=matRes[,c("DI","Lower","Upper")],
               labels=data.frame("Model"=rownames(matRes),"Accuracy"=matRes[,"Accuracy"]),
               points.pch=15,points.cex=3,points.col=rainbow(6),
               values=FALSE,xlim=c(0.1,1),lwd=4,cex=1.5,
               xlab="Disparate Impact",xlab.cex=1,xratio=0.3,y.title.offset=1)

**Q** Comparer les différents bais en tenant compte de l'intersection des intervalles de confiance? Quels sont les algorithmes qui discriminent significativement? Est-il efficace de retirer la variable sensible du modèle? Le post-processing sur les décisions des forêts aléatoires est-il efficace? Que dire du *testing*?

## 6 Autres indicateurs de biais / discrimination
### 6.1 Biais sur les erreurs ou précisions de prévision conditionnelles
L'effet disproportionné est une première source de biais ou de discrimination parmi d'autres. Une deuxième, souvent évoquée, concerne les erreurs de prévision ou les précisions selon les modalités de la variable sensible; c'est  l'*overall error equality* ou de façon équivalente l'*overall accuracy equality*.

#### Régression logistique linéaire
*Overall error equality*

In [ ]:
table(pred.log>0.5,daTest$income,daTest$sex)

In [ ]:
apply(table(pred.log>0.5,daTest$income,daTest$sex),3,tauxErr)


**Q** Quel genre semble défavorisé au regard de ce critère?

In [ ]:
source('../Functions/overErrEqual.R')

In [ ]:
round(overErrEqual(daTest$sex,daTest$income,as.factor(pred.log>0.5)),2)

**Q** Même question.

#### Random forest

In [ ]:
apply(table(yHatrf,daTest$income,daTest$sex),3,tauxErr)

In [ ]:
round(overErrEqual(daTest$sex,daTest$income,yHatrf),2)

**Q** Quel genre semble défavorisé au regard de ce critère?

#### *Random forest* avec discrimination positive

In [ ]:
tauxErr(table(yHatFrfDP,daTestF$income)); tauxErr(table(yHatMrfDP,daTestM$income))

In [ ]:
round(overErrEqual(daTestFM[,"sex"],daTestFM$income,as.factor(yHatFMrfDP)),2)

**Q** L'évolution de ce critère est elle logique compte tenu de la correction adoptée sur la décision?

### 6.2 Asymétrie de la matrice de confusion: *equalitzed odds* 
Une autre source de discrimination est considére. Elle a été mise en exergue par le site [Propublica](https://www.propublica.org/article/machine-bias-risk-assessments-in-criminal-sentencing) à propos du score de récidive COMPAS de la société *equivant* utilisé dans certaines cours de justice américaines sous une forme de "justice prédictive". Le biais concerne une inversion de l'asymétrie de la matrice de confusion selon la variable sensible. De très nombreux critères ont été proposés pour évaluer cette asymétrie et dont [Verma et Rubin (2018](http://fairware.cs.umass.edu/papers/Verma.pdf)) proposent une revue. Leurs définitions sont basées sur les différentes fréquences issues de la table de contingence et calculées par la fonction ci-dessous.

In [ ]:
contRatio <- function(T){ 
    # Ratios calculés à partir de la matrice de confusion
    TP=T[2,2] # true positive
    FP=T[2,1] # false positive
    FN=T[1,2] # false negative
    TN=T[1,1]  # true negative
    PPV=TP/(TP+FP) # P(Y=1|g(x)=1) positive predictive value
    FDR=FP/(TP+FP) # P(Y=0|g(x)=1) false discovery rate 
    FOR=FN/(TN+FN) # P(Y=1|g(x)=0) false omission rate
    NPV=TN/(TN+FN) # P(Y=0|g(x)=0) negative predictive value
    TPR=TP/(TP+FN) # P(g(x)=1|Y=1) true positive rate
    FPR=FP/(FP+TN) # P(g(x)=1|Y=0) false positive rate
    FNR=FN/(TP+FN) # P(g(x)=0|Y=1) false negative rate
    TNR=TN/(FP+TN) # P(g(x)=0|Y=0) true negative rate
    return(list("PPV"=PPV,"FDR"=FDR,"FOR"=FOR,"NPV"=NPV,"TPR"=TPR,"FPR"=FPR,"FNR"=FNR,"TNR"=TNR))
}

In [ ]:
contRatio(table(pred.log>0.5,daTest$income))

Liste de critères d'équité pouvant être définis à partir des fréquences précédentes conditionnellement à la variable sensible. La combinatoire des possibles est importante mais peut être réduite en remarquant que *PPV*=1-*FDR*, *FOR*=1-*NPV*, *FPR*=1-*TNR*, *FNR*=1-*TPR*... Selon les auteurs, il y a équité de traitement si:
- *Predictive parity*: les deux groupes ont les mêmes *PPV*s et par conséquence les mêmes *FDR*s;
- *False positive error rate balance* ou *predictive equality*: mêmes *FPR*s et par conséquence les mêmes *TNR*s;
- *False negative error rate balance* ou *equal opportunity*: mêmes *FNR*s et par conséquence les mêmes *TPR*s;
- *Conditional procedure accuracy equality* ou *disparate mistreatment* ou *equalized odds* combine les deux ci-dessus: mêmes *TPR*s **ET** mêmes *FPR*s;
- *Overall accuracy equality*: mêmes *TPR* **ET** mêmes *TNR*;
- *Conditionnal use accuracy equality*: mêmes *PPV*s **ET** mêmes *NPV*s;
- *Teatment equality*: les rapports *FN/FP* sont les mêmes pour les deux groupes.

Bien d'autres critères ont été proposés (cf. Verma et Rubin. 2018), il ne sont pas développés ici. Les calculs des *TPR* et *TNR* conditionnels sont privilégiés ci-dessous mais ce n'est qu'un choix parmi d'autres. Friedler et al. (2019) montrent que ceux-ci sont très corrélés. Il serait raisonnable de se limiter aux seules comparaisons des *TPR* et *FPR*.

#### Régression logistique linéaire
La matrice de confusion est construite pour chaque genre pour comparer les différents indicateurs de loyauté.

In [ ]:
fairness=data.frame("Female"=as.matrix(contRatio(table(pred.log>0.5,daTest$income,daTest$sex)[,,1])),
                    "Male"=as.matrix(contRatio(table(pred.log>0.5,daTest$income,daTest$sex)[,,2])))
fairness

Il serait fastidieux de construire toutes les comparaisons d'autant que beaucoup de ces indicateurs sont redondants. Seuls les  ***Equalized Odds*** sont estimés par des intervalles de  confiance à l'aide de la fonciton `oddsEqual` qui  admet 4 paramètres:
- S: variable du groupe protégé
- Y: variable cible
- P: prévision $\hat{Y}$
- alpha=0.05, valeur par défaut.

Elle fournit l'estimation par intervalle de confiance des rapports des *FPR* et *TPR* conditionnels et permet donc de tester l'égalité ou non des cotes selon la variable sensible.

In [ ]:
source('../Functions/oddsEqual.R')

In [ ]:
round(oddsEqual(daTest$sex,daTest$income,as.factor(pred.log>0.5)),2)

**Q** Quel genre semble cette fois favorisé au sens de ce critère?

#### *Random forest* 

In [ ]:
fairnessRF=data.frame("Female"=as.matrix(contRatio(table(yHatrf,daTest$income,daTest$sex)[,,1])),
                    "Male"=as.matrix(contRatio(table(yHatrf,daTest$income,daTest$sex)[,,2])))
fairnessRF

In [ ]:
round(oddsEqual(daTest$sex,daTest$income,yHatrf),2)

**Q** Même question avec une forêt aléatoire.

#### *Random forest* avec discrimination positive

In [ ]:
fairnessRFDP=data.frame("Female"=as.matrix(contRatio(table(yHatFrfDP,daTestF$income))),
                    "Male"=as.matrix(contRatio(table(yHatMrfDP,daTestM$income))))
fairnessRFDP

In [ ]:
round(oddsEqual(daTestFM[,"sex"],daTestFM$income,as.factor(yHatFMrfDP)),2)

**Q** La correction va-t-elle dans le sens attendu? 

**Q** En conclusion, le traitement par post-processing du seuil va-t-il, pour cet exemple, dans le bon sens pour tous les critères de biais?

## Conclusion

Conclure sur 
- le choix entre un modèle linéaire ou non linéaire, 
- l'intervention de la variable sensible dans un modèle et donc sur l'effet d'une interdiction de prendre en compte une variable sensible comme l'origine ethnique,
- l'efficacité du testing, 
- celle du post processing rudimentaire mis en oeuvre,
- l echoix entre un modèle "interprétable" mais moins précis et les forêts aléatoires...

**Remarques** .
- Il n'est sans doute pas nécessaire d'estimer deux modèles de forêts aléatoires selon le genre. Le seul post-processing des seuils de décision devrait suffire.
- Les entreprises proposant du recrutement prédictif en France ne se préoccupent pas de ces "détails", allant jusqu'au déni des risques de discrimination,
- contrairement à ce qui se passe aux USA où le calcul de l'*adverse* ou *disparate impact* est pris en compte par obligation du code du travail en tenant à jour des "statistiques ethniques".
- Aux USA, l'utilisation d'un algorithme non linéaire sans contrôle est risquée car un biais trop important ($DI<0.8$) sans explication, donc justification possible, est condamnable. C'est la raison pour laquelle certaines entreprises américaines de la *hiring tech* proposent des atténuations du biais afin d'économiser des poursuites judiciaires (Raghavan et al. 2019).
- Un `data scientist` dispose actuellement d'une grande latitude pour faire ce qu'il veut, sans contrôle: du comportement déloyal, condamnable, à la discrimination positive pour introduire plus d'équité dans la société!
- Nous pouvons espérer, qu'après la mise en place du RGPD, la Commission Européenne intervienne rapidement sur ces questions. C'est en tous cas le sens des directives annoncées dans le [livre blanc](https://ec.europa.eu/info/sites/info/files/commission-white-paper-artificial-intelligence-feb2020_fr.pdf).
- Un gros travail en vue pour un *data scientist* responsable d'un traitement qui devra tenir à jour une documentation complète et détaillée (liste d'évaluation) de toutes ces procédures, du recueil des données au suivi d'un système d'intelligence artificielle en exploitation. 

**Il est vivement recommandé d'anticiper sur cette réglementation à venir afin de justifier les choix opérés, à savoir le moins mauvais compromis faisant intervenir, confidentialité des données, précision des prévision, interprétabilité du modèle et biais ou risques de discrimination.**

## Références
Barocas S., Selbst A. (2016). [Big Data's Disparate Impact](https://papers.ssrn.com/sol3/papers.cfm?abstract_id=2477899), *California Law Review*, 671.

Besse P. (2020). [Détecter, évaluer les risques des impacts discriminatoires des algorithmes d'IA](https://hal.archives-ouvertes.fr/hal-02616963), Contribution au séminaire Défenseur des Droits et CNIL, 28 mai 2020, article soumis.

Besse P., del Barrio E., Gordaliza P., Loubes J.-M. (2018). [Confidence Intervals for testing Disparate Impact in Fair Learning](https://arxiv.org/abs/1807.06362), arXiv preprint.

Efron B. (1987). [Better Bootstrap Confidence Intervals](https://www.jstor.org/stable/2289144?seq=1), *Journal of the American Statistical Association*, Vol. 82, No. 397 (Mar., 1987), pp. 171-185. 

Friedler S., Scheidegger C., Venkatasubramanian S., Choudhary S., Hamilton E., Roth D. (2019). [A comparative study of fairness-enhancing interventions in machine learning](https://dl.acm.org/doi/10.1145/3287560.3287589), *Proceedings of the Conference on Fairness, Accountability, and Transparency*.

Morris S., Lobsenz R. (2000), [Signifiance Tests and Confidence Intervals for the Adverse Impact Ratio](https://doi.org/10.1111/j.1744-6570.2000.tb00195.x) *Personnel Psychology*, 53: 89-111.

Riach, P., Rich J. (2002). [Field Experiments of Discrimination In The Market Place]( https://doi.org/10.1111/1468-0297.00080), *The Economic Journal*, Vol. 112, 480-518.

Raghavan M., Barocas S., Kleinberg J., Levy K. (2019) [Mitigating bias in Algorithmic Hiring : Evaluating Claims and Practices](https://arxiv.org/abs/1906.09208), arXiv:1906.09208.

Verma S., Rubin J. (2018). [Fairness Definitions Explained](http://fairware.cs.umass.edu/papers/Verma.pdf),  *ACM/IEEE International Workshop on Software Fairness*.

Zliobaitė I. (2015). [A survey on measuring indirect discrimination in machine learning](https://arxiv.org/pdf/1511.00148.pdf), arXiv preprint.
